# Grid search exploration for optimal SVC parameters

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import keras
import json
import pandas as pd
from sklearn.svm import SVC
from path import Path
from sklearn.utils.class_weight import compute_class_weight
from path import Path
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import SCORERS

2023-04-11 19:51:21.358464: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 19:51:27.824103: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 19:51:27.825663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 19:52:07.456859: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
base_path = Path("/nfs/turbo/seas-nhcarter/human_wildlife_interactions/classifier_video_data")

In [2]:
# bring in the data
train_path = Path(base_path / "train_mat.csv")
test_path = Path(base_path / "test_mat.csv")
val_path = Path(base_path / "val_mat.csv")
train_df = pd.read_csv(train_path) 
test_df = pd.read_csv(test_path)
val_df = pd.read_csv(val_path)

In [3]:
train_df.shape, test_df.shape, val_df.shape

((2794, 1154), (494, 1154), (581, 1154))

In [4]:
X_train = train_df.iloc[:,:-1]
y_train = train_df.iloc[:, -1]
X_val = val_df.iloc[:,:-1]
y_val = val_df.iloc[:,-1]
X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1]

In [5]:
test_clf = SVC(random_state=42).fit(X_train, y_train)

In [6]:
test_clf.score(X_val,y_val)

0.9156626506024096

In [7]:
test_clf.score(X_test, y_test)

0.9149797570850202

In [8]:
weights = compute_class_weight('balanced',np.unique(y_train), y_train)

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2789    0.0
2790    1.0
2791    0.0
2792    0.0
2793    0.0
Name: 1152, Length: 2794, dtype: float64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [12]:
weighted_clf = SVC(random_state=42, class_weight='balanced',kernel='sigmoid').fit(X_train, y_train)

precision_recall_fscore_support(y_val, weighted_clf.predict(X_val))

(array([0.90340909, 0.07901235]),
 array([0.29887218, 0.65306122]),
 array([0.44915254, 0.14096916]),
 array([532,  49]))

In [17]:
parameters = {"C": [.001, .01, .1, 1, 10, 100],
             "kernel": ['linear', 'rbf', 'sigmoid'],
             "gamma": ['scale', 'auto'],
             "class_weight": [None, 'balanced', {0:.1,1:.9},{0:.08,1:.92},{0:.09,1:.91}],
             "random_state":[42]}
svc = SVC()
clf = GridSearchCV(svc, parameters,scoring='roc_auc', verbose=3)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END C=0.001, class_weight=None, gamma=scale, kernel=linear, random_state=42;, score=0.764 total time=   8.1s
[CV 2/5] END C=0.001, class_weight=None, gamma=scale, kernel=linear, random_state=42;, score=0.813 total time=  12.1s
[CV 3/5] END C=0.001, class_weight=None, gamma=scale, kernel=linear, random_state=42;, score=0.854 total time=  14.7s
[CV 4/5] END C=0.001, class_weight=None, gamma=scale, kernel=linear, random_state=42;, score=0.779 total time=  11.3s
[CV 5/5] END C=0.001, class_weight=None, gamma=scale, kernel=linear, random_state=42;, score=0.750 total time=   8.9s
[CV 1/5] END C=0.001, class_weight=None, gamma=scale, kernel=rbf, random_state=42;, score=0.557 total time=   0.7s
[CV 2/5] END C=0.001, class_weight=None, gamma=scale, kernel=rbf, random_state=42;, score=0.580 total time=   0.7s
[CV 3/5] END C=0.001, class_weight=None, gamma=scale, kernel=rbf, random_state=42;, score=0.390 total time=   0.7s
[C

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced', {0: 0.1, 1: 0.9},
                                          {0: 0.08, 1: 0.92},
                                          {0: 0.09, 1: 0.91}],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid'],
                         'random_state': [42]},
             scoring='roc_auc', verbose=3)

In [18]:
clf.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'C': [0.001, 0.01, 0.1, 1, 10, 100],
  'kernel': ['linear', 'rbf', 'sigmoid'],
  'gamma': ['scale', 'auto'],
  'class_weight': [None,
   'balanced',
   {0: 0.1, 1: 0.9},
   {0: 0.08, 1: 0.92},
   {0: 0.09, 1: 0.91}],
  'random_state': [42]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': 'roc_auc',
 'verbose': 3}

In [19]:
clf.best_params_

{'C': 0.01,
 'class_weight': {0: 0.1, 1: 0.9},
 'gamma': 'scale',
 'kernel': 'linear',
 'random_state': 42}

In [20]:
clf.best_estimator_

SVC(C=0.01, class_weight={0: 0.1, 1: 0.9}, kernel='linear', random_state=42)